In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch

from model.CringeLDM import CringeLDM
from PIL import Image

from utils import convert_to_rgb, import_image_from_path

/home/huey/miniconda3/envs/teeny/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the model
modelWrapper = CringeLDM()

# Import an image
img = import_image_from_path()

# Load the image
res = modelWrapper.forward(query="I am a cat", x=img, steps=1);

# Convert the image to RGB
res = convert_to_rgb(res)

# Show the image
plt.imshow(res)

In [4]:
from data.unsplashlite import UnsplashLiteDataset
import torch.utils.data
from torch.utils.data import DataLoader
import pytorch_lightning as pl

# data
dataset = UnsplashLiteDataset(root_dir='/mnt/e/Source/unsplash-lite-corpus-preprocess/db')
training_set, validation_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

train_loader = DataLoader(training_set, batch_size=32)
val_loader = DataLoader(validation_set, batch_size=32)

# model
model = CringeLDM()

# training
# trainer = pl.Trainer(accelerator='gpu', precision=16, limit_train_batches=0.5)
# trainer.fit(model, train_loader, val_loader)

for idx in range(0, len(train_loader)):
    text, image = train_loader.dataset[idx] 
    model.training_step((text, image), idx)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: too many values to unpack (expected 2)